In [1]:
from skimage.draw import polygon, ellipse_perimeter
from skimage.morphology import * 
import nibabel as nb
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
import numpy as np
import pandas as pd

In [2]:
def add_nodule(img, center, r, noise, orientation, k_dil, k_ero, coff, var ):
    rr, cc = ellipse_perimeter(center[0], center[1], r[0], r[1], orientation)
    rr = rr + np.random.normal(0, noise[0], len(rr))
    cc = cc + np.random.normal(0, noise[1], len(cc))
    x_list, y_list = polygon(rr, cc)
    no_img = np.zeros(img.shape, dtype = np.uint8)
    no_img[x_list, y_list] = 255
    img_dil = dilation(no_img, disk(k_dil))
    img_ero = erosion(img_dil, disk(k_ero))
    coors = np.mgrid[:img.shape[0], :img.shape[1]]
    distance_map = (coors[0] - center[0]) ** 2 + ((coors[1] - center[1])) ** 2 #  / r[1] * r[0]
    heat_map = coff* np.exp(-distance_map / var)
    mask = img_ero * heat_map
    new_img = img + mask
    new_mask = 255 * np.exp(-distance_map / 1000.0)
    return new_mask, new_img

def pos_csv():
    r1 = np.random.normal(7, 2)
    r2 = r1 + np.random.normal(0, 1)
    r1, r2 = int(abs(r1)), int(abs(r2))
    n1 = np.random.normal(1.5,0.5)
    n2 = n1 + np.random.normal(0, 0.5)
    n1, n2 = abs(n1), abs(n2)
    orientation = np.random.normal(0,1)
    k_dil = np.random.normal(1.5, 1)
    k_ero= np.random.normal(1.5, 1)
    k_dil = 1 + abs(k_dil - 1)
    k_ero = 1 + abs(k_ero - 1)
    coff = np.random.normal(0.8, 0.2)
    coff = max(coff, 0.2)
    coff = min(1, coff)
    var = np.random.normal(60, 30)
    var = max(20, var)
    return [r1,r2], ['%.2f'%n1,'%.2f'%n2],'%.2f'%orientation,'%.2f'%k_dil, '%.2f'%k_ero, '%.2f'%coff, '%.2f'%var, True

def neg_csv():
    r1 = np.random.normal(4, 1)
    r2 = r1 + np.random.normal(0, 0.8)
    r1, r2 = int(max(r1, 1)), int(max(r2,1))
    n1 = np.random.normal(0.8,0.5)
    n2 = n1 + np.random.normal(0, 0.2)
    n1, n2 = abs(n1), abs(n2)
    orientation = np.random.normal(0,1)
    k_dil = np.random.normal(1, 0.5)
    k_ero= np.random.normal(1, 0.5)
    k_dil = 1 + abs(k_dil - 1)
    k_ero = 1 + abs(k_ero - 1)
    coff = np.random.uniform(0.6, 0.2)
    coff = max(coff, 0.2)
    coff = min(1, coff)
    var = np.random.normal(50, 20)
    var = max(10, var)
    return [r1,r2], ['%.2f'%n1,'%.2f'%n2],'%.2f'%orientation,'%.2f'%k_dil, '%.2f'%k_ero, '%.2f'%coff, '%.2f'%var, False

#def nodule2scan(npy, info, center, r, noise, orientation, k_dil, k_ero, coff, var):
def get_slice_center(npy):
    num_slice = npy.shape[0]
    slice_list = []
    center_list = []
    #print (num_slice, 'num_slice')
    while True:
        flag = 0
        index = np.random.randint(10, num_slice - 10)
        thresh = threshold_otsu(npy[index])
        binary = npy[index] <= thresh
        for i in range(5):
            c0 = np.random.randint(5, npy[index].shape[0] -5)
            c1 =np.random.randint(5, npy[index].shape[1] - 5)
            if binary[c0, c1] == 1:
                flag = 1
                break
        if flag == 1:
            slice_list.append(index)
            center_list.append([c0, c1])
        if len(slice_list) >= 5:
            break
    return slice_list, center_list


In [ ]:
df = pd.read_csv('/nfs/masi/NLST/file/csv_file/label_sumulation.csv')
data_root = '/nfs/masi/NLST/DSB_File/no_diag/prep/prep_noreport_link'
all_slice, all_center = [], []
for i, item in df.iterrows():
    try:
        npy = np.load(data_root + '/' + item['sess'] + '_clean.npy')[0]
        slice_list, center_list = get_slice_center(npy)
        all_slice.append(slice_list)
        all_center.append(center_list)
    except: 
        all_slice.append('')
        all_center.append("")
df['slice'] = all_slice
df['center'] = all_center
df.to_csv('/nfs/masi/NLST/file/csv_file/label_sumulation2.csv', index = False)
    #print (slice_list, center_list)
    

In [ ]:
from skimage.filters import threshold_otsu
thresh = threshold_otsu(img_nii[0])
binary = img_nii[0] <= thresh
while True:
    center = np.random.randint(256, size = 2)
    print ('--')
    if binary[center[0], center[1]] == 1:
        break
print (center, binary[center[0], center[1]], img_nii.shape)

In [ ]:
clean_npy = np.load('/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/tmp/9954615106time20100101_clean.npy')
img_nii = nb.load(img_path).get_data()
img = clean_npy[0][140]
mask, new_img = add_nodule(img, center = [71, 40], r = [10, 9], noise = [0.8,0.3], orientation = -0.5, k_dil = 1.8,k_ero= 1.8, coff= 0.7, var = 30)
plt.imshow(mask, 'gray')
plt.show()
print (new_img[50][50])

In [5]:
from skimage import transform
import nibabel as nib
import re
df = pd.read_csv('/nfs/masi/NLST/file/csv_file/label_sumulation_neg.csv')
df = df[:10000]
data_root = '/nfs/masi/NLST/DSB_File/no_diag/prep/prep_noreport_link'
save_root = '/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/NLST/2Dnorm'
sample_size = 224
def info2float(info):
    info[0] = int(info[0])
    info[1] = int(info[1])
    info[2] = float(info[2])
    info[3] = float(info[3])
    info[4] = float(info[4])
    info[5] = float(info[5])
    info[6] = float(info[6])
    info[7] = float(info[7])
    info[8] = float(info[8])
    return info
    
#def center2int(center):
    
    
for i, item in df.iterrows():
    try:
        if i % 50 == 0: print (i, len(df))
        try:
            npy = np.load(data_root + '/' + item['sess'] + '_clean.npy')[0]
        except:
            print (item['sess'] + ' not existed')
            continue
        img_2D = np.zeros((5, sample_size, sample_size), dtype = np.uint8)
        mask_2D = np.zeros((5, sample_size, sample_size), dtype = np.uint8)
        info_vec = re.split('[(,\' )\[\]]', item['info'])
        info_vec = [i for i in info_vec if len(i) > 0]
        center_vec = re.split('[, \[\]]', item['center'])
        center_vec = [i for i in center_vec if len(i) > 0]
        slice_vec = re.split('[, \[\]]', item['slice'])
        slice_vec = [i for i in slice_vec if len(i) > 0]
        #print (item['center'])
        for i in range(item['num_nodule']):
            #print (info_vec)
            info = info2float(info_vec[10*i: 10 * i + 9])
            #print (item['slice'][i])
            new_mask, new_img = add_nodule(npy[int(slice_vec[i])], center = [int(center_vec[2*i]), int(center_vec[2*i + 1])], r = [info[0], info[1]], noise = [info[2], info[3]], 
                                       orientation = info[4], k_dil = info[5],k_ero= info[6], coff= info[7], var = info[8])
            img_2D[i] = transform.resize(new_img, [sample_size, sample_size], mode='edge', preserve_range='True')
            mask_2D[i] = transform.resize(new_mask, [sample_size, sample_size], mode='edge', preserve_range='True')
        nii_img = nib.Nifti1Image(img_2D, affine = np.eye(4))
        nii_mask = nib.Nifti1Image(mask_2D, affine = np.eye(4))    
        nib.save(nii_img, save_root + '/img/' + item['sess'] + '.nii.gz')
        nib.save(nii_mask, save_root + '/mask/' + item['sess'] + '.nii.gz') 
    except: 
        continue
    

0 10000
100037time1999 not existed
100047time2001 not existed
100061time1999 not existed
100081time1999 not existed
100099time2000 not existed
100099time1999 not existed
100117time1999 not existed
100129time1999 not existed
100143time2000 not existed
100156time2001 not existed
100230time1999 not existed
100310time1999 not existed
100344time2000 not existed
100347time1999 not existed
100349time1999 not existed
100357time2000 not existed
100361time1999 not existed
100374time2000 not existed
100389time1999 not existed
100392time2001 not existed
100436time1999 not existed
100441time1999 not existed
100458time2001 not existed
50 10000
100494time1999 not existed
100519time2000 not existed
100519time1999 not existed
100543time1999 not existed
100547time1999 not existed
100576time2001 not existed
100646time1999 not existed
100675time1999 not existed
100689time1999 not existed
100690time1999 not existed
100710time1999 not existed
100712time1999 not existed
100716time1999 not existed
100734time1

106502time2000 not existed
106577time2000 not existed
106593time2000 not existed
106633time2000 not existed
106674time2000 not existed
106675time1999 not existed
106773time2000 not existed
106713time1999 not existed
106728time1999 not existed
106792time1999 not existed
106793time1999 not existed
106823time2001 not existed
106825time1999 not existed
106848time1999 not existed
106866time2000 not existed
106877time1999 not existed
106880time2001 not existed
106887time2000 not existed
106887time1999 not existed
106900time2001 not existed
106906time2001 not existed
650 10000
106949time2000 not existed
106954time2000 not existed
106954time1999 not existed
106955time1999 not existed
106959time1999 not existed
106963time2000 not existed
106963time1999 not existed
106984time2001 not existed
106991time1999 not existed
107015time2001 not existed
107041time1999 not existed
107049time2001 not existed
107053time2000 not existed
107053time1999 not existed
107057time1999 not existed
107088time2001 not

112341time2000 not existed
112355time2000 not existed
112355time1999 not existed
112384time1999 not existed
112428time2001 not existed
112428time2000 not existed
112439time1999 not existed
1200 10000
112445time2000 not existed
112464time2001 not existed
112464time1999 not existed
112467time1999 not existed
112483time2000 not existed
112513time1999 not existed
112520time1999 not existed
112525time2000 not existed
112525time1999 not existed
112608time2000 not existed
112621time1999 not existed
112640time2001 not existed
112649time1999 not existed
112667time1999 not existed
112701time1999 not existed
112758time2000 not existed
112798time2000 not existed
112818time2000 not existed
112818time1999 not existed
112847time2001 not existed
112875time2001 not existed
112881time1999 not existed
112918time1999 not existed
112947time1999 not existed
112998time2001 not existed
113017time1999 not existed
113059time1999 not existed
1250 10000
113049time2000 not existed
113125time2001 not existed
113175

118205time2000 not existed
118205time1999 not existed
118253time2001 not existed
118278time1999 not existed
118289time2000 not existed
118343time2000 not existed
118343time1999 not existed
118353time1999 not existed
118359time2001 not existed
118453time1999 not existed
118465time1999 not existed
118525time2001 not existed
118530time2001 not existed
118550time2001 not existed
118569time1999 not existed
118571time1999 not existed
118582time2001 not existed
118588time1999 not existed
118591time1999 not existed
118599time1999 not existed
1850 10000
118613time2001 not existed
118663time2000 not existed
118675time1999 not existed
118685time2000 not existed
118697time2001 not existed
118700time2000 not existed
118761time1999 not existed
118775time2001 not existed
118775time1999 not existed
118788time2000 not existed
118805time2000 not existed
118846time1999 not existed
118860time1999 not existed
118889time1999 not existed
118895time2000 not existed
118913time1999 not existed
118916time1999 no

124078time1999 not existed
2450 10000
124120time1999 not existed
124121time1999 not existed
124161time1999 not existed
124197time1999 not existed
124199time2001 not existed
124208time1999 not existed
124209time1999 not existed
124221time1999 not existed
124225time1999 not existed
124257time2000 not existed
124296time1999 not existed
124306time1999 not existed
124317time1999 not existed
124321time1999 not existed
124394time2001 not existed
124473time1999 not existed
124474time1999 not existed
124485time1999 not existed
124493time1999 not existed
124495time2000 not existed
124496time2000 not existed
124506time2001 not existed
124506time1999 not existed
124508time1999 not existed
124531time2000 not existed
124535time1999 not existed
124564time2000 not existed
124572time1999 not existed
2500 10000
124586time2000 not existed
124610time1999 not existed
124621time2001 not existed
124684time2001 not existed
124686time1999 not existed
124701time2001 not existed
124717time2000 not existed
124743

...

109217time1999 not existed
109217time2000 not existed
109217time2001 not existed
109219time2000 not existed
109226time2000 not existed
109230time2000 not existed
109231time2001 not existed
109233time2000 not existed
109289time2000 not existed
109289time2001 not existed
109301time2000 not existed
109302time1999 not existed
109302time2001 not existed
109306time2001 not existed
8000 10000
109337time2000 not existed
109355time2001 not existed
109363time2001 not existed
109382time1999 not existed
109382time2000 not existed
109393time2000 not existed
109393time2001 not existed
109397time1999 not existed
109397time2000 not existed
109397time2001 not existed
109399time1999 not existed
109399time2000 not existed
109399time2001 not existed
109400time1999 not existed
109400time2000 not existed
109401time1999 not existed
109401time2000 not existed
109401time2001 not existed
109417time2001 not existed
109419time1999 not existed
109419time2000 not existed
109419time2001 not existed
109421time2000 no

111154time1999 not existed
111154time2000 not existed
111154time2001 not existed
111166time1999 not existed
111166time2000 not existed
111166time2001 not existed
111175time1999 not existed
111184time2000 not existed
111189time1999 not existed
111189time2000 not existed
111232time1999 not existed
111232time2000 not existed
8500 10000
111254time2001 not existed
111256time1999 not existed
111256time2000 not existed
111256time2001 not existed
111270time1999 not existed
111270time2000 not existed
111287time2001 not existed
111303time2001 not existed
111304time1999 not existed
111315time1999 not existed
111338time1999 not existed
111341time2000 not existed
111341time2001 not existed
111343time1999 not existed
111343time2000 not existed
111369time1999 not existed
111369time2000 not existed
111378time1999 not existed
111383time2001 not existed
111387time2000 not existed
111390time1999 not existed
111391time2000 not existed
111416time1999 not existed
111416time2001 not existed
111443time1999 no

113226time2000 not existed
113227time2001 not existed
113232time1999 not existed
113232time2000 not existed
113232time2001 not existed
113238time2001 not existed
113241time1999 not existed
113241time2001 not existed
113245time1999 not existed
113255time2001 not existed
113257time1999 not existed
113261time1999 not existed
113261time2000 not existed
113261time2001 not existed
113266time1999 not existed
113266time2000 not existed
113266time2001 not existed
113272time2000 not existed
113272time2001 not existed
113274time1999 not existed
113279time1999 not existed
113279time2000 not existed
113290time2000 not existed
113294time2001 not existed
113301time1999 not existed
113326time1999 not existed
113343time1999 not existed
113343time2000 not existed
113343time2001 not existed
113344time1999 not existed
113344time2000 not existed
113345time1999 not existed
113345time2000 not existed
113345time2001 not existed
9000 10000
113352time2000 not existed
113360time2000 not existed
113371time1999 no

115232time2001 not existed
115242time2000 not existed
115254time2001 not existed
115262time2000 not existed
115262time2001 not existed
115270time1999 not existed
115288time2001 not existed
115292time1999 not existed
115292time2001 not existed
115299time2001 not existed
115306time1999 not existed
115306time2001 not existed
115320time1999 not existed
115320time2000 not existed
115335time1999 not existed
115335time2000 not existed
115335time2001 not existed
115338time2001 not existed
115342time2001 not existed
115344time1999 not existed
115344time2000 not existed
115346time2000 not existed
115346time2001 not existed
115347time1999 not existed
115347time2000 not existed
115353time2001 not existed
115363time2000 not existed
9500 10000
115377time2000 not existed
115378time2000 not existed
115401time2000 not existed
115433time1999 not existed
115433time2001 not existed
115440time1999 not existed
115448time1999 not existed
115448time2001 not existed
115452time2001 not existed
115467time1999 no

117117time2000 not existed
117122time2001 not existed
117143time1999 not existed
117143time2000 not existed
9950 10000
117143time2001 not existed
117146time2000 not existed
117149time1999 not existed
117149time2001 not existed
117157time1999 not existed
117157time2000 not existed
117157time2001 not existed
117163time2000 not existed
117198time1999 not existed
117198time2001 not existed
117203time1999 not existed
117206time2000 not existed
117210time2001 not existed
117214time1999 not existed
117214time2000 not existed
117214time2001 not existed
117223time1999 not existed
117223time2000 not existed
117223time2001 not existed
117250time2000 not existed
117256time1999 not existed
117266time1999 not existed
117266time2001 not existed
117268time2001 not existed
117277time1999 not existed
117277time2000 not existed
117277time2001 not existed
117288time2001 not existed
117297time2000 not existed
117297time2001 not existed
117298time1999 not existed
117298time2000 not existed
117308time1999 no

In [ ]:
clean_npy = np.load('/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/tmp/9954615106time20100101_clean.npy')
print (clean_npy.shape)
slice_list, center_list = get_slice_center(clean_npy[0])
new_data = np.zeros((5, 224, 224), dtype = np.uint8)
#for i in range(5):
    

In [ ]:
max(int(np.random.normal(5, 3)), 2)
max(int(np.random.normal(2, 1)), 1)

In [ ]:
import pandas as pd
df = pd.read_csv('/nfs/masi/NLST/file/csv_file/label_noreport.csv')
num_nodule, num_posnod = [], []
for i, item in df.iterrows():
    num = min(5, max(int(np.random.normal(5, 3)), 2))
    num_nodule.append(num)
    if item['gt'] == 1:
        num_posnod.append(min(num, max(int(np.random.normal(2, 1)), 1)))
    else:
        num_posnod.append(0)
df['num_nodule'] = num_nodule
df['num_posnod'] = num_posnod
df.to_csv('/nfs/masi/NLST/file/csv_file/label_sumulation.csv', index = False)

In [ ]:
df = pd.read_csv('/nfs/masi/NLST/file/csv_file/label_sumulation.csv')
info = []
for i, item in df.iterrows():
    tmp_info = []
    for i in range(item['num_posnod']):
        tmp_info.append(pos_csv())
    for i in range(item['num_nodule'] - item['num_posnod']):
        tmp_info.append(neg_csv())
    info.append(tmp_info)
df['info'] = info
df.to_csv('/nfs/masi/NLST/file/csv_file/label_sumulation.csv', index = False)

In [ ]:
float('-1.2')

In [ ]:
import re
a = "[([4, 5], ['0.28', '0.08'], '0.33', '1.15', '1.01', '0.20', '92.52', False), ([5, 6], ['0.86', '0.96'], '-2.03', '1.21', '1.40', '0.31', '48.34', False), ([4, 4], ['0.37', '0.19'], '0.09', '1.13', '1.84', '0.29', '38.19', False), ([1, 2], ['0.11', '0.25'], '0.75', '1.27', '1.21', '0.23', '79.56', False), ([4, 3], ['0.28', '0.56'], '0.68', '2.09', '1.21', '0.37', '55.18', False)]"
a_vec = re.split('[(,\' )\[\]]', a)
new_a = [i for i in a_vec if len(i) > 0]
print (a_vec)
print (new_a)